<a href="https://colab.research.google.com/github/charlottieee/HelpfulScripts/blob/main/Booking_In_Storage_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date
import pandas as pd
from google.colab import files

# Create text boxes for each field
date_input = widgets.Textarea(description='Date:', value=date.today().strftime('%Y/%m/%d'))
barcode_input = widgets.Textarea(description='Barcode:')
sku_input = widgets.Textarea(description='SKU:')
item_code_input = widgets.Textarea(description='Item Code:')
quantity_input = widgets.Textarea(description='Quantity:')

# Create the Push to Excel Sheet button
push_button = widgets.Button(description="Push to Excel Sheet")

# Create the Add New Entry button
add_new_button = widgets.Button(description="Add New Entry")

# Create the Save Changes button (initially hidden)
save_changes_button = widgets.Button(description="Save Changes")
save_changes_button.layout.display = 'none' # Hide initially

# Create the Download Excel button
download_button = widgets.Button(description="Download Excel")


# Variable to store the index of the row being edited
editing_index = None

# Placeholder for displaying the DataFrame and edit/delete buttons
data_display_output = widgets.Output()

# Function to refresh the displayed data with Edit and Delete buttons
def refresh_display():
    try:
        df = pd.read_excel("inventory.xlsx")
    except FileNotFoundError:
        df = pd.DataFrame(columns=['Date', 'Barcode', 'SKU', 'Item Code', 'Quantity'])

    with data_display_output:
        clear_output(wait=True)

        if not df.empty:
            # Create a list of widgets for each row
            row_widgets = []
            # Add header row
            header_labels = [widgets.Label(value=col, layout=widgets.Layout(width='100px')) for col in df.columns]
            header_labels.extend([widgets.Label(value="", layout=widgets.Layout(width='auto')), widgets.Label(value="", layout=widgets.Layout(width='auto'))]) # Placeholder for buttons
            header_hbox = widgets.HBox(header_labels)
            row_widgets.append(header_hbox)


            for index, row in df.iterrows():
                # Create HBox for each row containing data, edit, and delete buttons
                edit_button = widgets.Button(description="Edit", layout=widgets.Layout(width='auto'))
                edit_button.on_click(lambda b, index=index: edit_entry(index))

                delete_button = widgets.Button(description="Delete", layout=widgets.Layout(width='auto'), button_style='danger')
                delete_button.on_click(lambda b, index=index: delete_entry(index))


                # Create labels for each column's data
                date_label = widgets.Label(value=str(row['Date']), layout=widgets.Layout(width='100px'))
                barcode_label = widgets.Label(value=str(row['Barcode']), layout=widgets.Layout(width='100px'))
                sku_label = widgets.Label(value=str(row['SKU']), layout=widgets.Layout(width='100px'))
                item_code_label = widgets.Label(value=str(row['Item Code']), layout=widgets.Layout(width='100px'))
                quantity_label = widgets.Label(value=str(row['Quantity']), layout=widgets.Layout(width='100px'))

                # Arrange labels and buttons in an HBox
                row_hbox = widgets.HBox([date_label, barcode_label, sku_label, item_code_label, quantity_label, edit_button, delete_button])
                row_widgets.append(row_hbox)

            # Display the rows
            display(widgets.VBox(row_widgets))
        else:
            display(widgets.Label(value="No data in inventory."))

# Define the function to handle Push to Excel Sheet button clicks (Create)
def push_to_excel(button):
    global editing_index
    new_data = {
        'Date': [date_input.value],
        'Barcode': [barcode_input.value],
        'SKU': [sku_input.value],
        'Item Code': [item_code_input.value],
        'Quantity': [quantity_input.value]
    }
    new_row_df = pd.DataFrame(new_data)

    try:
        existing_df = pd.read_excel("inventory.xlsx")
    except FileNotFoundError:
        existing_df = pd.DataFrame(columns=['Date', 'Barcode', 'SKU', 'Item Code', 'Quantity'])

    updated_df = pd.concat([existing_df, new_row_df], ignore_index=True)
    updated_df.to_excel("inventory.xlsx", index=False)

    print("Data pushed to inventory.xlsx!")
    refresh_display() # Refresh display after adding

    # After pushing, reset to add new entry mode
    add_new_entry(None)
    save_changes_button.layout.display = 'none'
    push_button.layout.display = 'block'
    editing_index = None


# Define the function to handle Add New Entry button clicks
def add_new_entry(button):
    global editing_index
    date_input.value = date.today().strftime('%Y/%m/%d')
    barcode_input.value = ''
    sku_input.value = ''
    item_code_input.value = ''
    quantity_input.value = ''
    print("Input fields cleared for a new entry.")
    save_changes_button.layout.display = 'none'
    push_button.layout.display = 'block'
    editing_index = None


# Define the function to handle Edit button clicks (Update - populate fields)
def edit_entry(index):
    global editing_index
    try:
        df = pd.read_excel("inventory.xlsx")
        if index < len(df):
            row_to_edit = df.iloc[index]
            # Format the date when populating for editing
            date_input.value = pd.to_datetime(row_to_edit['Date']).strftime('%Y/%m/%d')
            barcode_input.value = str(row_to_edit['Barcode'])
            sku_input.value = str(row_to_edit['SKU'])
            item_code_input.value = str(row_to_edit['Item Code'])
            quantity_input.value = str(row_to_edit['Quantity'])

            editing_index = index # Store the index of the row being edited

            # Show the Save Changes button and hide the Push button
            save_changes_button.layout.display = 'block'
            push_button.layout.display = 'none'

            print(f"Editing row {index}")
        else:
            print(f"Error: Index {index} out of bounds.")
    except FileNotFoundError:
        print("Error: inventory.xlsx not found.")


# Define the function to handle Save Changes button clicks (Update - save changes)
def save_changes(button):
    global editing_index
    if editing_index is not None:
        try:
            df = pd.read_excel("inventory.xlsx")
            if editing_index < len(df):
                # Update the row with the new data from input fields
                df.loc[editing_index, 'Date'] = date_input.value
                df.loc[editing_index, 'Barcode'] = barcode_input.value
                df.loc[editing_index, 'SKU'] = sku_input.value
                df.loc[editing_index, 'Item Code'] = item_code_input.value
                df.loc[editing_index, 'Quantity'] = quantity_input.value

                # Save the updated DataFrame
                df.to_excel("inventory.xlsx", index=False)
                print(f"Changes saved for row {editing_index}")

                # Refresh the display
                refresh_display()

                # Reset the UI to add new entry mode
                add_new_entry(None)
                save_changes_button.layout.display = 'none'
                push_button.layout.display = 'block'
                editing_index = None
            else:
                print(f"Error: Editing index {editing_index} is no longer valid.")
        except FileNotFoundError:
            print("Error: inventory.xlsx not found.")
    else:
        print("No row is currently being edited.")

# Define the function to handle Delete button clicks (Delete)
def delete_entry(index):
    try:
        df = pd.read_excel("inventory.xlsx")
        if index < len(df):
            # Remove the row at the specified index
            df = df.drop(index).reset_index(drop=True)

            # Save the modified DataFrame
            df.to_excel("inventory.xlsx", index=False)
            print(f"Row {index} deleted.")

            # Refresh the display
            refresh_display()
        else:
            print(f"Error: Index {index} out of bounds for deletion.")
    except FileNotFoundError:
        print("Error: inventory.xlsx not found.")

# Define the function to handle Download button clicks
def download_excel(button):
    try:
        files.download("inventory.xlsx")
        print("Downloading inventory.xlsx")
    except FileNotFoundError:
        print("Error: inventory.xlsx not found. Please add some data first.")


# Link the buttons to their functions
push_button.on_click(push_to_excel)
add_new_button.on_click(add_new_entry)
save_changes_button.on_click(save_changes)
download_button.on_click(download_excel) # Link download button

# Display the text boxes, buttons, and data display area
display(date_input)
display(barcode_input)
display(sku_input)
display(item_code_input)
display(quantity_input)
display(push_button)
display(add_new_button)
display(save_changes_button)
display(download_button) # Display the download button
display(data_display_output)


# Initial display of the data
refresh_display()

Textarea(value='2025-09-25', description='Date:')

Textarea(value='', description='Barcode:')

Textarea(value='', description='SKU:')

Textarea(value='', description='Item Code:')

Textarea(value='', description='Quantity:')

...

Button(description='Download Excel', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Row 1 deleted.
Row 0 deleted.
